In [1]:
import pandas as pd
import logging

from split_data_type import SplitDataType
from pipeline import TreeSimilarPipeline
from feature_manager import FeatureManager

In [2]:
logging.basicConfig(level=logging.DEBUG)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
target_column = "SalePrice"

train_df = pd.read_csv("../HousePrice_train.csv")
train_X = train_df.drop(columns=[target_column])
train_y = train_df[target_column]
test_X = pd.read_csv("../HousePrice_test.csv")
test_y = pd.read_csv("../sample_submission.csv")[target_column]

In [4]:
FeatureManager.get_features_with_none(train_X)

'LotFrontage, Alley, MasVnrType, MasVnrArea, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Electrical, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature'

In [5]:
FeatureManager.get_smallest_category(train_X)

,value,frequency(%)
column,,
Utilities,NoSeWa,0.068493
RoofMatl,Metal,0.068493
Exterior1st,AsphShn,0.068493
Condition2,PosA,0.068493
HeatingQC,Po,0.068493
Heating,Floor,0.068493
ExterCond,Po,0.068493
Exterior2nd,Other,0.068493
Functional,Sev,0.068493


In [6]:
FeatureManager.get_high_entropy(train_X)

['Id']

# Досліджую фічі, де є маленький % значень та заповнюю нулі

In [7]:
# заповню середнім арифметичним
print(train_X['LotFrontage'].isnull().sum())
train_X['LotFrontage'].value_counts().head()

259


LotFrontage
60.0    143
70.0     70
80.0     69
50.0     57
75.0     53
Name: count, dtype: int64

In [8]:
# в with_alley
print(train_X['Alley'].isnull().sum())
train_X['Alley'].value_counts() 

1369


Alley
Grvl    50
Pave    41
Name: count, dtype: int64

In [9]:
# в with masonry
print(train_X['MasVnrType'].isnull().sum())
train_X['MasVnrType'].value_counts() 

872


MasVnrType
BrkFace    445
Stone      128
BrkCmn      15
Name: count, dtype: int64

In [10]:
# заповню середнім арифметичним
print(train_X['MasVnrArea'].isnull().sum())
train_X['MasVnrArea'].value_counts().head()

8


MasVnrArea
0.0      861
108.0      8
72.0       8
180.0      8
16.0       7
Name: count, dtype: int64

In [11]:
# оцінічне
print(train_X['BsmtQual'].isnull().sum())
train_X['BsmtQual'].value_counts()

37


BsmtQual
TA    649
Gd    618
Ex    121
Fa     35
Name: count, dtype: int64

In [12]:
# оцінічне
print(train_X['BsmtCond'].isnull().sum())
train_X['BsmtCond'].value_counts()

37


BsmtCond
TA    1311
Gd      65
Fa      45
Po       2
Name: count, dtype: int64

In [13]:
# оцінічне
print(train_X['BsmtExposure'].isnull().sum())
train_X['BsmtExposure'].value_counts()

38


BsmtExposure
No    953
Av    221
Gd    134
Mn    114
Name: count, dtype: int64

In [14]:
# в is_finished_bsmt_fin_type(1, 2)
print(train_X['BsmtFinType1'].isnull().sum())
train_X['BsmtFinType1'].value_counts()
print(train_X['BsmtFinType2'].isnull().sum())
train_X['BsmtFinType2'].value_counts()

37
38


BsmtFinType2
Unf    1256
Rec      54
LwQ      46
BLQ      33
ALQ      19
GLQ      14
Name: count, dtype: int64

In [15]:
# в is_standard_electrical
print(train_X['Electrical'].isnull().sum())
train_X['Electrical'].value_counts() 

1


Electrical
SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: count, dtype: int64

In [16]:
# оцінка
print(train_X['FireplaceQu'].isnull().sum())
train_X['FireplaceQu'].value_counts()

690


FireplaceQu
Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: count, dtype: int64

In [17]:
# в attchd, detchd, others
print(train_X['GarageType'].isnull().sum())
train_X['GarageType'].value_counts()

81


GarageType
Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: count, dtype: int64

In [18]:
# -1 бо нема гаража
print(train_X['GarageYrBlt'].isnull().sum())
train_X['GarageYrBlt'].value_counts().head()

81


GarageYrBlt
2005.0    65
2006.0    59
2004.0    53
2003.0    50
2007.0    49
Name: count, dtype: int64

In [19]:
# No garage бо нема гаража
print(train_X['GarageFinish'].isnull().sum())
train_X['GarageFinish'].value_counts()

81


GarageFinish
Unf    605
RFn    422
Fin    352
Name: count, dtype: int64

In [20]:
# оцінічне
print(train_X['GarageQual'].isnull().sum())
train_X['GarageQual'].value_counts() 

81


GarageQual
TA    1311
Fa      48
Gd      14
Ex       3
Po       3
Name: count, dtype: int64

In [21]:
# оцінічне
print(train_X['GarageCond'].isnull().sum())
train_X['GarageCond'].value_counts() 

81


GarageCond
TA    1326
Fa      35
Gd       9
Po       7
Ex       2
Name: count, dtype: int64

In [22]:
# в with pool
print(train_X['PoolQC'].isnull().sum())
train_X['PoolQC'].value_counts() 

1453


PoolQC
Gd    3
Ex    2
Fa    2
Name: count, dtype: int64

In [23]:
# оцінка
print(train_X['Fence'].isnull().sum())
train_X['Fence'].value_counts()

1179


Fence
MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: count, dtype: int64

In [24]:
# в with feature
print(train_X['MiscFeature'].isnull().sum())
train_X['MiscFeature'].value_counts()

1406


MiscFeature
Shed    49
Gar2     2
Othr     2
TenC     1
Name: count, dtype: int64

In [25]:
# Мусор фіча, в смітник
train_X['Utilities'].value_counts() 

Utilities
AllPub    1459
NoSeWa       1
Name: count, dtype: int64

In [26]:
# Мусор фіча
train_X['RoofMatl'].value_counts() 

RoofMatl
CompShg    1434
Tar&Grv      11
WdShngl       6
WdShake       5
Metal         1
Membran       1
Roll          1
ClyTile       1
Name: count, dtype: int64

In [27]:
# перетворю < 200 в others
train_X['Exterior1st'].value_counts() 

Exterior1st
VinylSd    515
HdBoard    222
MetalSd    220
Wd Sdng    206
Plywood    108
CemntBd     61
BrkFace     50
WdShing     26
Stucco      25
AsbShng     20
BrkComm      2
Stone        2
AsphShn      1
ImStucc      1
CBlock       1
Name: count, dtype: int64

In [28]:
# мусорка
train_X['Condition2'].value_counts() 

Condition2
Norm      1445
Feedr        6
Artery       2
RRNn         2
PosN         2
PosA         1
RRAn         1
RRAe         1
Name: count, dtype: int64

In [29]:
# Зведу всі оціночні категорії в positive, neutral, negative
train_X['HeatingQC'].value_counts() 

HeatingQC
Ex    741
TA    428
Gd    241
Fa     49
Po      1
Name: count, dtype: int64

In [30]:
# мусор
train_X['Heating'].value_counts() 

Heating
GasA     1428
GasW       18
Grav        7
Wall        4
OthW        2
Floor       1
Name: count, dtype: int64

In [31]:
train_X['ExterCond'].value_counts() 

ExterCond
TA    1282
Gd     146
Fa      28
Ex       3
Po       1
Name: count, dtype: int64

In [32]:
# < 100 в others
train_X['Exterior2nd'].value_counts() 

Exterior2nd
VinylSd    504
MetalSd    214
HdBoard    207
Wd Sdng    197
Plywood    142
CmentBd     60
Wd Shng     38
Stucco      26
BrkFace     25
AsbShng     20
ImStucc     10
Brk Cmn      7
Stone        5
AsphShn      3
Other        1
CBlock       1
Name: count, dtype: int64

In [33]:
# в is_typical_functional
train_X['Functional'].value_counts() 

Functional
Typ     1360
Min2      34
Min1      31
Mod       15
Maj1      14
Maj2       5
Sev        1
Name: count, dtype: int64

In [34]:
# в is_standard_electrical
train_X['Electrical'].value_counts() 

Electrical
SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: count, dtype: int64

In [35]:
# нічого не роблю
train_X['Neighborhood'].value_counts()

Neighborhood
NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR      37
ClearCr     28
SWISU       25
StoneBr     25
Blmngtn     17
MeadowV     17
BrDale      16
Veenker     11
NPkVill      9
Blueste      2
Name: count, dtype: int64

In [36]:
# в is_gable_roof_style
train_X['RoofStyle'].value_counts() 

RoofStyle
Gable      1141
Hip         286
Flat         13
Gambrel      11
Mansard       7
Shed          2
Name: count, dtype: int64

In [37]:
# в is_norm_condition1
train_X['Condition1'].value_counts() 

Condition1
Norm      1260
Feedr       81
Artery      48
RRAn        26
PosN        19
RRAe        11
PosA         8
RRNn         5
RRNe         2
Name: count, dtype: int64

In [38]:
# в new, warrante_deed, others
train_X['SaleType'].value_counts() 

SaleType
WD       1267
New       122
COD        43
ConLD       9
ConLI       5
ConLw       5
CWD         4
Oth         3
Con         2
Name: count, dtype: int64

In [39]:
# < 200 в others
train_X['Foundation'].value_counts() 

Foundation
PConc     647
CBlock    634
BrkTil    146
Slab       24
Stone       6
Wood        3
Name: count, dtype: int64

In [40]:
# в is_normal_sale_condition
train_X['SaleCondition'].value_counts() 

SaleCondition
Normal     1198
Partial     125
Abnorml     101
Family       20
Alloca       12
AdjLand       4
Name: count, dtype: int64

In [41]:
# в inside, corner, others
train_X['LotConfig'].value_counts() 

LotConfig
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: count, dtype: int64

In [42]:
# мусор фіча
train_X['Street'].value_counts()

Street
Pave    1454
Grvl       6
Name: count, dtype: int64

In [43]:
# звужую до 1 этажний, 2 этажний, інші
train_X['HouseStyle'].value_counts()

HouseStyle
1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
1.5Unf     14
2.5Unf     11
2.5Fin      8
Name: count, dtype: int64

In [44]:
# в is_reg
train_X['LotShape'].value_counts()

LotShape
Reg    925
IR1    484
IR2     41
IR3     10
Name: count, dtype: int64

In [45]:
# в is_residential
train_X['MSZoning'].value_counts()

MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: count, dtype: int64

In [46]:
# в is_gtl
train_X['LandSlope'].value_counts()

LandSlope
Gtl    1382
Mod      65
Sev      13
Name: count, dtype: int64

In [47]:
# оцінічне
train_X['ExterQual'].value_counts()

ExterQual
TA    906
Gd    488
Ex     52
Fa     14
Name: count, dtype: int64

In [48]:
# в is_paved
train_X['PavedDrive'].value_counts()

PavedDrive
Y    1340
N      90
P      30
Name: count, dtype: int64

In [49]:
# нічого не роблю
train_X['BldgType'].value_counts()

BldgType
1Fam      1220
TwnhsE     114
Duplex      52
Twnhs       43
2fmCon      31
Name: count, dtype: int64

In [50]:
# в is_level
train_X['LandContour'].value_counts()

LandContour
Lvl    1311
Bnk      63
HLS      50
Low      36
Name: count, dtype: int64

In [51]:
train_X['MasVnrType'].value_counts()

MasVnrType
BrkFace    445
Stone      128
BrkCmn      15
Name: count, dtype: int64

In [52]:
# оцінка
train_X['KitchenQual'].value_counts()

KitchenQual
TA    735
Gd    586
Ex    100
Fa     39
Name: count, dtype: int64

In [53]:
# в is_central_air
train_X['CentralAir'].value_counts()

CentralAir
Y    1365
N      95
Name: count, dtype: int64

In [54]:
# оцінка
train_X['BsmtExposure'].value_counts()

BsmtExposure
No    953
Av    221
Gd    134
Mn    114
Name: count, dtype: int64

In [55]:
#raise Exception("Stop here to check the features")

In [56]:
train_X = TreeSimilarPipeline(train_X, SplitDataType.TRAIN).build()

DEBUG:root:TRAIN - Function: _drop_not_needed
Size: (1460, 74)
Columns with None: LotFrontage, Alley, MasVnrType, MasVnrArea, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Electrical, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature

DEBUG:root:TRAIN - Function: _fill_null
Size: (1460, 74)
Columns with None: BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Electrical, FireplaceQu, GarageType, GarageQual, GarageCond, Fence

DEBUG:root:TRAIN - Function: _encode
Size: (1460, 74)
Columns with None: None

DEBUG:root:TRAIN - Function: _normalize
Size: (1460, 74)
Columns with None: None

DEBUG:root:TRAIN - Function: _drop_high_correlation
Size: (1460, 74)
Columns with None: None



In [57]:
train_X.head()

,MSSubClass,LotFrontage,LotArea,LotConfig,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,WithAlley,WithMasonry,WithPool,WithFeature,Is_finished_bsmt_fintype1,Is_finished_bsmt_fintype2,Is_typical_functional,Is_standard_electrical,Is_gable_roofstyle,Is_norm_condition1,Is_normal_sale_condition,Is_reg_lotshape,Is_residential_mszoning,Is_Gtl_landslope,Is_paved,Is_level_landContour,Is_central_air
0,60,65.0,8450,Inside,CollgCr,1Fam,2Story,7,5,2003,2003,VinylSd,VinylSd,196.0,positive,neutral,PConc,positive,neutral,Others,706,0,150,856,positive,856,854,0,1710,1,0,2,1,3,1,positive,8,0,Others,Attchd,2003.0,RFn,2,548,neutral,neutral,0,61,0,0,0,0,Others,0,2,2008,Warranty_deed,0,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1
1,20,80.0,9600,Others,Veenker,1Fam,1Story,6,8,1976,1976,MetalSd,MetalSd,0.0,neutral,neutral,CBlock,positive,neutral,positive,978,0,284,1262,positive,1262,0,0,1262,0,1,2,0,3,1,neutral,6,1,neutral,Attchd,1976.0,RFn,2,460,neutral,neutral,298,0,0,0,0,0,Others,0,5,2007,Warranty_deed,0,0,0,0,1,0,1,1,1,0,1,1,1,1,1,1,1
2,60,68.0,11250,Inside,CollgCr,1Fam,2Story,7,5,2001,2002,VinylSd,VinylSd,162.0,positive,neutral,PConc,positive,neutral,negative,486,0,434,920,positive,920,866,0,1786,1,0,2,1,3,1,positive,6,1,neutral,Attchd,2001.0,RFn,2,608,neutral,neutral,0,42,0,0,0,0,Others,0,9,2008,Warranty_deed,0,1,0,0,1,0,1,1,1,1,1,0,1,1,1,1,1
3,70,60.0,9550,Corner,Crawfor,1Fam,2Story,7,5,1915,1970,Wd Sdng,Others,0.0,neutral,neutral,Others,neutral,positive,Others,216,0,540,756,positive,961,756,0,1717,1,0,1,0,3,1,positive,7,1,positive,Detchd,1998.0,Unf,3,642,neutral,neutral,0,35,272,0,0,0,Others,0,2,2006,Warranty_deed,0,0,0,0,1,0,1,1,1,1,0,0,1,1,1,1,1
4,60,84.0,14260,Others,NoRidge,1Fam,2Story,8,5,2000,2000,VinylSd,VinylSd,350.0,positive,neutral,PConc,positive,neutral,neutral,655,0,490,1145,positive,1145,1053,0,2198,1,0,2,1,4,1,positive,9,1,neutral,Attchd,2000.0,RFn,3,836,neutral,neutral,192,84,0,0,0,0,Others,0,12,2008,Warranty_deed,0,1,0,0,1,0,1,1,1,1,1,0,1,1,1,1,1
